In [1]:
!nvidia-smi

Sat Dec 12 09:26:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Dependencies

In [2]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
import torch
import joblib
import os

#tf denpendencies
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Activation,Input,Bidirectional
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.utils import plot_model,to_categorical

#plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [3]:
if torch.cuda.is_available():      
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [8]:
#install streamlit
!pip install streamlit

In [5]:
#install pyngrok
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp36-none-any.whl size=15970 sha256=62cd0c034fcd325d2c824555805f40b8a24905968486d1c661fea2d0ac03e5cf
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
#install plotly
!pip install plotly==4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [10]:
#install scikit-tensor
!pip install scikit-tensor-py3

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Streamlit

In [11]:
%%writefile utils.py
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
import torch
import joblib
import os
import random

#tf denpendencies
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM,Dropout,Activation,Input,Bidirectional
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.models import load_model,save_model
from tensorflow.keras.utils import plot_model,to_categorical
from sktensor import dtensor


def load_dataset(df_file_path,route_dir,user_dir):
  # Load Raw Data
  cols = ['id', 'userId', 'name', 'city', 'gender', 'sport',
          'calories', 'Route_id', 'distance_adjusted_sum','altitude_adjusted','distance_adjusted','speed_adjusted','heart_rate_adjusted','derived_speed','longitude','latitude']
  df = pd.read_csv(df_file_path, usecols=cols)
  df.rename(columns={'speed_adjusted': 'speed','altitude_adjusted':'altitude','distance_adjusted':'derived_distance','heart_rate_adjusted':'heart_rate'},inplace=True)
  scaler_dic = dict()
  features = ['calories','altitude','derived_distance','speed','heart_rate','distance']
  # Load Min-max scaler models
  dir = '/content/gdrive/My Drive/COMP5703/scaler_model'
  # https://drive.google.com/drive/folders/1Q9act2dsQ6lfau-PYw7pXVKVlO4Q22IB?usp=sharing

  for feature in features:
      path = os.path.join(dir, 'scaler_'+feature+'_2.m')
      scaler_dic[feature] = joblib.load(path)
  # Load onehot encoder for categorical features

  dir = '/content/gdrive/My Drive/COMP5703/onehot/OneHotEncoder.m'
  # https://drive.google.com/file/d/1wKtDVj4jxk1ViLj1lhzwdXIn82aNAc9h/view?usp=sharing

  OneHot_enc = joblib.load(dir)

  user_embed_df = joblib.load(user_dir)
  route_embed_df = joblib.load(route_dir)

  return df,user_embed_df,route_embed_df,OneHot_enc,scaler_dic


def data_process(gender, sport, calories, Route_id, distance_adjusted_sum, user_embed, route_embed, scaler_dic,OneHot_enc):
    # label encode gender

    choices = [0, 1, 2]
    conditions = [
        (gender == 'male'),
        (gender == 'female'),
        (gender == 'unknown')]

    genderId = np.select(conditions, choices, default=0)

    # label encode sport

    conditions = [
        (sport == 'run'),
        (sport == 'bike'),
        (sport == 'mountain bike')]

    sportId = np.select(conditions, choices, default=0)

    # scale calories
    calories_scaled = scaler_dic['calories'].transform(
        np.array(calories).reshape(-1, 1))[0]
    # scale distance
    total_distance_scaled = scaler_dic['distance'].transform(
        np.array(distance_adjusted_sum).reshape(-1, 1))[0]
    # one hot encode genderId and sportId
    gender_sport_onehot = OneHot_enc.transform(
        np.hstack((genderId, sportId)).reshape(1, -1)).toarray()[0]
    # concatenate input features into numpy array
    data_input = np.hstack((calories_scaled, total_distance_scaled,
                            user_embed, route_embed, gender_sport_onehot))

    return data_input

def predict_distance(model,data_input,scaler_dic,device):
  model.eval()
  with torch.no_grad():
      pred = model(torch.Tensor(data_input).to(device)).item()

  # convert predicted distance to km
  pred_km = scaler_dic['distance'].inverse_transform(
      np.array(pred).reshape(-1, 1))[0][0]
  return pred_km

def get_stop_point(pred_distance,derived_distance):
  # get where should the workout stop according to the calculated distance from model 1
  # if the predicted distance is larger than the whole workout distance, return -1
  if pred_distance > sum(derived_distance):
    return -1
  cur = 0
  for i in range(len(derived_distance)):
    cur += derived_distance[i]
    if cur >= pred_distance:
      return i

def generate_workout(df,pred_distance,workout_id,user_id,gender,calories,sport):
  # generate pd.DataFrame based on the information
  # get route
  temp = df[df['id'] == workout_id].copy()
  temp['userId'] = user_id
  temp['gender'] = gender
  temp['calories'] = calories
  temp['sport'] = sport
  temp.reset_index(drop=1,inplace=True)
  if len(temp) != 1:
    return None
  
  # Sport Gender Encoding
  choices = [0, 1, 2]
  conditions = [
      (temp['gender'] == 'male'),
      (temp['gender'] == 'female'),
      (temp['gender'] == 'unknown')]

  temp['genderId'] = np.select(conditions, choices, default=0)
  conditions = [
      (temp['sport'] == 'run'),
      (temp['sport'] == 'bike'),
      (temp['sport']  == 'mountain bike')]
  temp['sportId'] = np.select(conditions, choices, default=0)
  # handle contextual data
  context_info = ['id','userId','genderId','calories','sportId','Route_id']
  context_dict = {col:temp[col][0] for col in context_info}

  # handle sequence data
  seq = []
  speed = eval(temp['speed'][0])
  altitude = eval(temp['altitude'][0])
  distance = eval(temp['derived_distance'][0])
  hr = eval(temp['heart_rate'][0])

  complete_id = len(speed)
  if  complete_id <499:
    sub = 499-complete_id
    speed.extend([0]*sub)
    hr.extend([0]*sub)

  seq = [[altitude[i],distance[i],hr[i],speed[i]] for i in range(499)]
  sequence = np.array(seq)
  context = np.array([[context_dict['id'],context_dict['userId'],context_dict['genderId'],context_dict['sportId'],context_dict['Route_id'],context_dict['calories']]]*499)
  array = np.concatenate((context,
                sequence),axis=1)
  columns=['workoutId','userId','genderId','sportId','Route_id','calories','altitude','derived_distance','heart_rate','speed']
  df_sub = pd.DataFrame(array,columns=columns)
  # get stop point
  stop_point = get_stop_point(pred_distance,distance)
  return df_sub, stop_point

def convert_category(value,length):
  array_list = [0]*length
  array_list[int(value)] = 1
  return [array_list]*499

def generate_scaled(df_sub,user_embed,route_embed,scaler_dic):
  # generate scaled data based on workout dataframe, and return inputs and outputs for model2
  features = ['calories','altitude','derived_distance','speed','heart_rate']
  gender = convert_category(df_sub['genderId'][0],3)
  sport = convert_category(df_sub['sportId'][0],3)
  # route = [route_embed_dict[int(df_sub['Route_id'][0])]]*499
  # user = [user_embed_dict[int(df_sub['userId'][0])]]*499
  route = route_embed.reshape(1,-1).repeat(499,axis=0)
  user = user_embed.reshape(1,-1).repeat(499,axis=0)
  feature_array = np.concatenate((gender,sport,route,user),axis=1)
  for k in features:
    # speed input
    if k == 'speed':
      speed = scaler_dic['speed'].transform(df_sub[k].values.reshape(-1,1))
    # heart rate input
    elif k == 'heart_rate': 
      hr = scaler_dic['heart_rate'].transform(df_sub[k].values.reshape(-1,1))
    else:
      array = scaler_dic[k].transform(df_sub[k].values.reshape(-1,1))
      feature_array = np.concatenate((feature_array,array),axis=1)
  x1 = feature_array
  y1 = speed
  y2 = hr
  return x1.reshape(1,x1.shape[0],x1.shape[1]),y1.reshape(y1.shape[0],1),y2.reshape(y2.shape[0],1)

def apply_stop(stop_stamp):
  return np.append(np.ones(stop_stamp),np.zeros(499-stop_stamp))

def predict_sp_hr(df,user_embed_df,route_embed_df,scaler_dic,OneHot_enc,device,
                  distance_model,SpeedHr_model,
                    workout_id,user_id,gender,calories,sport):
  # combine model_1 and model_2

  # find Route_id based on workout id
  Route_id = df.loc[df.id == workout_id, 'Route_id'].to_numpy()[0]
  # find route total distance based on workout id
  distance_adjusted_sum = df.loc[df.id == workout_id, 'distance_adjusted_sum'].to_numpy()[0]
  # find userEmbedding based on userId
  user_embed = np.array(
      user_embed_df[user_embed_df.userId == user_id].userEmbed.values[0])
  # find routeEmbedding based on routeId
  route_embed = np.array(
      route_embed_df[route_embed_df.Route_id == Route_id].routeEmbed.values[0])
  # generate inputs for model_1
  input4distance = data_process(gender,sport,calories,Route_id,distance_adjusted_sum,user_embed,route_embed,scaler_dic,OneHot_enc)
  # predict distance
  pred_dis = predict_distance(distance_model,input4distance,scaler_dic,device)
  # generate workout according to the predicted distance
  input_df,stop_point =generate_workout(df,pred_distance=pred_dis,workout_id=workout_id,user_id=user_id,gender=gender,sport=sport,calories=calories)
  # return -1 if the distance is too short for the target input
  if stop_point == -1:
    return -1,-1,pred_dis,stop_point
  
  # Otherwise, generate input for model2
  input4sp, y_sp, y_hr = generate_scaled(input_df,user_embed,route_embed,scaler_dic)
  # predict speed and distance
  pred_sp,pred_hr = SpeedHr_model(input4sp)
  # inverse transform speed and distance & combine with the stop point
  pred_sp = scaler_dic['speed'].inverse_transform([np.array(pred_sp).flatten()]) * apply_stop(stop_point)
  pred_hr = scaler_dic['heart_rate'].inverse_transform([np.array(pred_hr).flatten()]) * apply_stop(stop_point)
  return pred_sp,pred_hr,pred_dis,stop_point



def load_model_torch(filename, model):

    DATA_PATH = "/content/gdrive/My Drive/COMP5703/onehot/"+filename
    # https://drive.google.com/drive/folders/1IQ7IoQV6PZXRj0G-CKJKBTMENsYRFI86?usp=sharing
    checkpoint = torch.load(DATA_PATH)
    model.load_state_dict(checkpoint['best_model_state_dict'])

    return model

# MLP model
def load_distance_model(LOAD_MODEL_NAME,device):
  class DisReg_MLP_2Layer(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, p):
        super(DisReg_MLP_2Layer, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, 1)

        self.act_1 = nn.ReLU()
        self.act_2 = nn.Sigmoid()

        self.drop_1 = nn.Dropout(p)

    def forward(self, data):

        output = self.fc1(data)
        output = self.act_1(output)
        output = self.drop_1(output)

        output = self.fc2(output)
        output = self.act_2(output)

        return output
  # LOAD_MODEL_NAME = 'DisReg_MLP_2'
  INPUT_DIM = 34
  HIDDEN_DIM = 64
  DROP_OUT = 0.2

  model = DisReg_MLP_2Layer(
    INPUT_DIM, HIDDEN_DIM, DROP_OUT).to(device)
  model = load_model_torch(LOAD_MODEL_NAME, model)
  return model

def load_sphr_model(filename):
  SpeedHr_model = load_model(filename)
  return SpeedHr_model


Writing utils.py


In [12]:
%%writefile app.py
import streamlit as st
#plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
# import torch.nn as nn
import torch
# from sktensor import dtensor
import pandas as pd
# import os
from utils import predict_sp_hr,load_dataset,load_distance_model,load_sphr_model
# from tensorflow.keras.models import load_model
import numpy as np
from time import strftime, gmtime
PAGE_CONFIG = {"page_title":"StColab.io","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)
import matplotlib

# Find the exact route on the Google map using plotly
def route_fig(df,route_option):
  #plot route
  longitude = eval(df[df['id']==route_option]['longitude'].values[0])
  latitude = eval(df[df['id']==route_option]['latitude'].values[0])
  df_new = pd.DataFrame({'longitude':longitude,'latitude':latitude}).reset_index()
  df_new['size'] = 4
  df_new.loc[0,'size'] = 15
  df_new.loc[499,'size'] = 15
  df_new['text'] = ''
  df_new.loc[0,'text'] = 'Start'
  df_new.loc[499,'text'] = 'End'
  
  # print(type(df_new))
  token = 'pk.eyJ1IjoicHVtcGtpbmt1biIsImEiOiJja2Y2a3puN3YwMTF1MnFucnJyNGRxaTJrIn0.c9VFzhysl-4MLPibge4P3A' # You can create your own token and use it
  fig_route = px.scatter_mapbox(df_new, lat="latitude", lon="longitude",color="index",size='size',text='text',size_max=12,
                                labels = {'index':'StartToEnd'},zoom = 11,color_continuous_scale=px.colors.sequential.Sunsetdark) 
  fig_route.update_layout(height=600,width=800,
    legend = dict(font=dict(size=1)),
    font=dict(size=24),
      mapbox = {
      'accesstoken': token,
      'style': "outdoors"})
  return fig_route



# main
if __name__ == '__main__':
  # load data and model 
  if torch.cuda.is_available():      
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')
  # path
  file_path = '/content/gdrive/My Drive/COMP5703/Dataset/UI_data.csv'
  # https://drive.google.com/file/d/1W8-eEOnEnjafqv-X1FMoozWnAFA5tVvA/view?usp=sharing
  route_dir = '/content/gdrive/My Drive/COMP5703/embedding/routeEmbed_tensorD_13.m'
  # https://drive.google.com/file/d/1oAvzIZEufHgEeIAUmxbwfnZmXG7Ragrt/view?usp=sharing
  user_dir= '/content/gdrive/My Drive/COMP5703/embedding/userEmbed_tensorD_13.m'
  # https://drive.google.com/file/d/1E8P_FqXgVzSAU-TwEFBSMU1OW694kSJu/view?usp=sharing
  LOAD_MODEL_NAME = 'DisReg_MLP_2L_13'
  model2_name = '/content/gdrive/My Drive/COMP5703/model/[1112]speed2hr_r13.h5'
  # https://drive.google.com/file/d/19DzgT4-eCFUvqFo-LbAaiKqSQUuOmkhl/view?usp=sharing
  df,user_embed_df,route_embed_df,OneHot_enc,scaler_dic = load_dataset(file_path,route_dir,user_dir)
  Distance_model = load_distance_model(LOAD_MODEL_NAME,device)
  SpeedHr_model = load_sphr_model(model2_name)
  
  ## GUI demo
  st.title("IoT Personalized Fitness Recommendation System")

  # User input interface
  st.sidebar.subheader("User name")
  user_option = st.sidebar.selectbox('',list(set(df['name'])))
  user_id = df[df['name']==user_option]['userId'].values[0]
  st.sidebar.subheader("Sport type")
  sport = st.sidebar.selectbox('',['run','bike','mountain bike'])
  st.sidebar.subheader("Workout route")
  workout_city = st.sidebar.selectbox('',df[df['sport']==sport]['city'].values)
  route_option = df[df['city']==workout_city]['id'].values[0]
  gender = df[df['name']==user_option]['gender'].values[0]
  recommend_cal = df[df['id']==route_option]['calories'].values[0]
  total_distance = df[df['id']==route_option]['distance_adjusted_sum'].values[0]
  st.subheader('Route overview on map')
  st.plotly_chart(route_fig(df,route_option))
  st.sidebar.subheader("Target calories")
  calories = st.sidebar.slider('',min_value = int(0.5*recommend_cal),max_value = int(2*recommend_cal),value=int(recommend_cal),step=1)
  if st.sidebar.button('Get In. Get Fit. Get on with Life!'):
    sp,hr,dis,stop_point = predict_sp_hr(df, user_embed_df, route_embed_df,scaler_dic,OneHot_enc,device,
                    Distance_model, SpeedHr_model, 
                      workout_id = route_option ,user_id=user_id,gender=gender,calories=calories,sport=sport)
    
    # If your target is hard to achieve
    if stop_point == -1:
      st.warning('Warning ! The required workout distance to achieve your calories target has exceeded the maximum distance of the selected workout route. Please reduce your calories target or select a different workout route. ')
    # Successful prediction
    else:
      # User input parameters summary
      st.sidebar.success('Predicted successfully')
      data = {
          'user name': user_option,
          'gender': gender,
          'workout id': workout_city,
          'sport_type': sport,
          'Target calories': calories
          }
      features = pd.DataFrame(data, index=[0])
      st.sidebar.subheader('User Input parameters')
      st.sidebar.write(features)
      st.write('The total length of the route you choose is:',total_distance, 'km')
      st.write('You need to exercise', round(dis, 2), 'km to achieve your fitness goals！')

      st.subheader('Predicted speed')
      dis_ls = df[df['id']==route_option]['derived_distance']
      dis_ls = eval(dis_ls.values[0])
      sp_ls = (np.array(sp.flatten())).tolist()
      hr_ls = (np.array(hr.flatten())).tolist()

      # find sequence length
      seq_len = len(hr_ls)
      # remove additional points from distance sequence
      dis_ls = dis_ls[0:seq_len]
      # compute minutes sequence
      min_ls = [(dis/sp_ls[idx])*60 if sp_ls[idx] != 0 else 0 for idx, dis in enumerate(dis_ls)]
      # interval to present in minutes
      interval = 5
      # new resampled minutes list
      min_new = []
      # original indices of each resampled chunck
      idx_ls = []
      start_idx = 0
      end_idx = 1
      # iterate over min_ls to resample data based on desired interval
      while end_idx <= seq_len:
          sum_min = sum(min_ls[start_idx:end_idx])

          if sum_min < 5 and end_idx < seq_len:
              end_idx += 1

          elif end_idx < seq_len:
              min_new.append(sum_min)
              idx_ls.append([start_idx, end_idx])
              start_idx = end_idx
              end_idx += 1

          else:
              # min_new.append(sum_min)
              # idx_ls.append([start_idx, end_idx])
              start_idx = end_idx
              end_idx += 1

      # compute new speed list based on resampled indices
      sp_new = [sum(dis_ls[indices[0]:indices[1]])/(min_new[i]/60) for i, indices in enumerate(idx_ls)]
      # round resampled minutes nearest number
      min_new_adjusted = [min//interval*interval for min in min_new]

      ## keep last item because it may not be a full interval period
      # min_new_adjusted[-1] = min_new[-1]

      # cumulative sum of resampled minutes list to be x axis
      min_new_adjusted_sum = list(np.cumsum(min_new_adjusted))

      # function to format minutes
      def format_time(min):
          return str(strftime("%H:%M:%S", gmtime(min*60)))

      # format x axis
      min_label = [format_time(min_new_adjusted_sum[i-1])+'-'+format_time(min_new_adjusted_sum[i]) if i>0 else format_time(0)+'-'+format_time(min_new_adjusted_sum[i]) for i in range(len(min_new_adjusted_sum))]

      # waterfall plot base value
      base = min(sp_new)
      # compute waterfall plot speed list based on base value
      sp_waterfall = [sp_new[i] - sp_new[i-1] if i>0 else sp_new[0]-base for i, _ in enumerate(sp_new)]
      # compute waterfall plot data labels
      sp_waterfall_label = ['+' + str(round(sp, 3)) if sp>0 else str(round(sp, 3)) for sp in sp_waterfall]
      # waterfall plot starting point should not be delta
      sp_waterfall_label[0] = str(round(sp_new[0], 3))

      # Speed line plot
      x_axis =np.arange(1,499)
      fig_sp = go.Figure()
      fig_sp.add_trace(go.Scatter(x=x_axis, y=sp.flatten(),
                    mode='lines'))
      fig_sp.update_layout(font_size=16,height=600,width=1400,
      xaxis = dict(title = 'TimeStamp'),
      yaxis = dict(title = 'Speed(km/h)'))
      st.plotly_chart(fig_sp)

      # Speed waterfall plot
      st.subheader('Speed recommendation')
      fig_sp_interval = go.Figure(go.Waterfall(
      orientation = 'v',
      x = min_label,
      textposition = 'outside',
      text = sp_waterfall_label,
      y = sp_waterfall,
      base = base,
      decreasing = {'marker':{'color': '#00bc12'}},
      increasing = {'marker':{'color': '#ff7500'}},
      connector = {'line':{'color': 'rgb(63, 63, 63)', 'width': 3}},
      ))

      fig_sp_interval.update_layout(font_size=16,height=600,width=1400,
              # title = 'Workout Speed Recommendation',
              xaxis_title= 'Time Elapsed (min)',
              yaxis_title= 'Speed (km/h)',
              yaxis=dict(range=[min(sp_new)*0.85, max(sp_new)*1.05])
      )

      st.plotly_chart(fig_sp_interval)

      # Heart-rate line plot
      st.subheader('Predicted heart rate')
      x_axis_hr = np.arange(1,stop_point)
      fig_hr = go.Figure()
      fig_hr.add_trace(go.Scatter(x=x_axis_hr, y=hr.flatten(),
                    mode='lines'))
      fig_hr.update_layout(font_size=16,height=600,width=1400,
      xaxis = dict(title = 'TimeStamp'),
      yaxis = dict(title = 'HeartRate(bpm)'))
      st.plotly_chart(fig_hr)

      # Heart-rate box plot
      st.subheader('Heart rate prediction')
      hr_new = [hr_ls[indices[0]:indices[1]] for indices in idx_ls]
      fig_hr_interval = go.Figure()

      # initialise the Matplotlib cmap & normalizer for boxplot colors
      norm = matplotlib.colors.Normalize(vmin=min(hr_ls), vmax=max(hr_ls))
      cmap = matplotlib.cm.get_cmap('RdYlBu')

      for i, hr in enumerate(hr_new):

          # find the median of each hr batch and use the normalizer to map it to an 
          # appropriate color based on the median value
          median = np.median(hr) # find the median
          color = 'rgb' + str(cmap(norm(median))[0:3]) # normalise
          
          fig_hr_interval.add_trace(go.Box(
              y=hr, 
              name=min_label[i], 
              fillcolor=color,
              line=dict(color='black')
              ))

      fig_hr_interval.update_layout(font_size=16,height=600,width=1400,
              # title = 'Workout Heart Rate Prediction',
              xaxis_title= 'Time Elapsed (min)',
              yaxis_title= 'Heart Rate (bpm)',
              showlegend = False
      )

      st.plotly_chart(fig_hr_interval)

Writing app.py


# Connection of Streamlit with URL by ngrok

In [13]:
# Check if the file has been written to the current Colab
!ls

app.py	gdrive	sample_data  utils.py


In [14]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [15]:
!streamlit run app.py &>/dev/null&

In [16]:
!pgrep streamlit

1161


In [17]:
# Use pyngrok and pass in the port of Streamlit (i.e. 8501)
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')

In [18]:
public_url

'http://a955489572d4.ngrok.io'

In [ ]:
# Stop it
!kill 741